General structure taken from Jon's A4 Notebook.

In [95]:
import matplotlib.pyplot as plt # needed for plotting
import numpy as np # numpy is primary library for numeric array (and matrix) handling
import scipy as sp
from scipy import stats, signal
import random
from sklearn import svm # needed for svm
from sklearn.metrics import confusion_matrix
import itertools
from scipy.io import wavfile
import librosa

from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis, skew

# Audio Capture

## Capture for Training

## Live Capture for Testing

# Data Processing and Classification

In [91]:
# Add preprocessing functions here if needed
def preprocess(rawSignal):
    
    # Add preprocessing functions here if needed
    return rawSignal

class LiveSound:
    def __init__(self, data, sampleRate):
        self.sound = data
        self.sound_float = data.astype(float)
        self.sound_p = preprocess(data)
        self.sample_rate = sampleRate
        
        self.soundClassification = None


class SoundTrial:
    def __init__(self, soundName, trialNum, filenameWithPath):
        self.soundName = soundName
        self.trialNum = trialNum
        self.filenameWithPath = filenameWithPath
        self.filename = os.path.basename(filenameWithPath)
        
        sampleRate, data = wavfile.read(filenameWithPath)
    
        self.sound = data
        self.sound_float = data.astype(float)
        self.sound_p = preprocess(data)
        self.sample_rate = sampleRate
    
    def __str__(self):
         return "'{}' : Trial {} from {}".format(self.soundName, self.trialNum, self.filename)
      
    
    
class SoundSet:
    def __init__(self, sound_sample_path, map_sounds_to_trials):
        self.path = sound_sample_path
        self.map_sounds_to_trials = map_sounds_to_trials 
        
    # returns the base path
    def get_base_path(self):
        return os.path.basename(os.path.normpath(self.path))
    
    # returns the number of sounds
    def get_num_sounds(self):
        return len(self.map_sounds_to_trials)
    
    # returns the total number of trials
    def get_total_num_of_trials(self):
        numTrials = 0 
        for soundName, trialSet in self.map_sounds_to_trials.items():
            numTrials = numTrials + len(trialSet)
        return numTrials
    
    # returns a sorted list of gesture names
    def get_sound_names_sorted(self):
        return sorted(self.map_sounds_to_trials.keys())
    

    # THESE HAVEN'T BEEN CHANGED YET ****************************
    # returns the longest trial (based on num rows recorded and not clock time)
    def get_longest_trial(self):
        longest_trial_length = -1
        longest_trial = None
        for gesture_name, trial_list in self.map_gestures_to_trials.items():
            for trial in trial_list:
                if longest_trial_length < len(trial.accel.x):
                    longest_trial_length = len(trial.accel.x)
                    longest_trial = trial
        return longest_trial
    

    
    # returns trials for a gesture name
    def get_trials_for_gesture(self, gesture_name):
        return self.map_gestures_to_trials[gesture_name]
    
    # creates an aggregate signal based on *all* trials for this gesture
    # TODO: in future could add in an argument, which takes a list of trial nums
    # to use to produce aggregate signal
    def create_aggregate_signal(self, gesture_name, signal_var_name):
        trials = self.get_trials_for_gesture(gesture_name)
        aggregate_signal = None
        trial_signals = []
        trial_signals_original = []
        first_trial = None
        first_trial_signal = None
        
        max_length = -1
        for trial in trials:
            trial_signal = getattr(trial.accel, signal_var_name)
            if max_length < len(trial_signal):
                max_length = len(trial_signal)
            
        for i in range(len(trials)):
            if i == 0:
                first_trial = trials[i]
                trial_signal = getattr(first_trial.accel, signal_var_name)
                trial_signal_mod = np.copy(trial_signal)

                trial_signals.append(trial_signal_mod)
                trial_signals_original.append(trial_signal)
                
                array_length_diff = max_length - len(trial_signal_mod)
                trial_signal_mod = np.pad(trial_signal_mod, (0, array_length_diff), 'mean')  

                aggregate_signal = trial_signal_mod
                first_trial_signal = trial_signal_mod
            else:

                cur_trial = trials[i]
                cur_trial_signal = getattr(trial.accel, signal_var_name) 
                trial_signals_original.append(cur_trial_signal)
                
                array_length_diff = max_length - len(cur_trial_signal)
                cur_trial_signal_mod = np.pad(cur_trial_signal, (0, array_length_diff), 'mean') 

                cur_trial_signal_mod = get_aligned_signal_cutoff_and_pad(cur_trial_signal_mod, first_trial_signal)
                trial_signals.append(cur_trial_signal_mod)
                aggregate_signal += cur_trial_signal_mod
        
        mean_signal = aggregate_signal / len(trial_signals) 
        return mean_signal

    # Returns the minimum number of trials across all gestures (just in case we accidentally recorded a 
    # different number. We should have the same number of trials across all gestures)
    def get_min_num_of_trials(self):
        minNumTrials = -1 
        for gestureName, trialSet in self.map_gestures_to_trials.items():
            if minNumTrials == -1 or minNumTrials > len(trialSet):
                minNumTrials = len(trialSet)
        return minNumTrials
    
    # get random gesture name
    def get_random_gesture_name(self):
        gesture_names = list(self.map_gestures_to_trials.keys())
        rand_gesture_name = gesture_names[random.randint(0, len(gesture_names) - 1)]
        return rand_gesture_name
    
    # get random trial
    def get_random_trial(self):
        rand_gesture_name = self.get_random_gesture_name()
        print("rand_gesture_name", rand_gesture_name)
        trials_for_gesture = self.map_gestures_to_trials[rand_gesture_name]
        return trials_for_gesture[random.randint(0, len(trials_for_gesture) - 1)]
    
    
    
    # prettify the str()
    def __str__(self):
         return "'{}' : {} sounds and {} total trials".format(self.path, self.get_num_sounds(), self.get_total_num_of_trials())
        

In [92]:
from os import listdir
import ntpath
import os

# From: https://stackoverflow.com/questions/800197/how-to-get-all-of-the-immediate-subdirectories-in-python
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

# Currently excludes any filenames with 'fulldatastream' in the title
def find_wav_filenames( path_to_dir, suffix=".wav" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix )]

def parse_and_create_sound_trials( path_to_dir ):
    wavFilenames = find_wav_filenames(path_to_dir)
    
    print("Found {} wav files in {}".format(len(wavFilenames), path_to_dir))
    
    mapSoundNameToTrialList = dict()
    mapSoundNameToMapSampleNum = dict()
    for wavFilename in wavFilenames:
        
        # parse filename into meaningful parts
        filenameNoExt = os.path.splitext(wavFilename)[0];
        filenameParts = filenameNoExt.split("_")
            
        soundName = filenameParts[0]
        sampleNum = filenameParts[1]
        fileName = "filename"
        #print("soundName={} sampleNum={}".format(soundName, sampleNum))
        
        
        if soundName not in mapSoundNameToMapSampleNum:
            mapSoundNameToMapSampleNum[soundName] = dict()
            
        if sampleNum not in mapSoundNameToMapSampleNum[soundName]:
            mapSoundNameToMapSampleNum[soundName][sampleNum] = dict()
            
        mapSoundNameToMapSampleNum[soundName][sampleNum][fileName] = wavFilename

        #print(mapSoundNameToMapSampleNum)
    
    print("Found {} sounds".format(len(mapSoundNameToMapSampleNum)))
   

    # Now we need to loop through the data and sort each sound set by timems values 
    # (so that we have trial 1, 2, 3, etc. in order)
    for soundName, mapSampleNumToFile in mapSoundNameToMapSampleNum.items():
        soundTrialNum = 0
        mapSoundNameToTrialList[soundName] = list()
        for sampleNum in sorted(mapSampleNumToFile.keys()):
            mapSampleToFile = mapSampleNumToFile[sampleNum]
            
            filenameWithPath = os.path.join(path_to_dir, mapSampleToFile["filename"])
            soundTrial = SoundTrial(soundName, soundTrialNum, filenameWithPath)
            mapSoundNameToTrialList[soundName].append(soundTrial)
            
            soundTrialNum = soundTrialNum + 1
        
        print("Found {} trials for '{}'".format(len(mapSoundNameToTrialList[soundName]), soundName))

    return mapSoundNameToTrialList


In [93]:
# Load the data
class MapSoundSets:
    def __init__(self,rootSoundSamplePath="./SoundSamples", targetDirWord="thresholdStop"):
        #sets map_sound_sets
        self.load_data(rootSoundSamplePath, targetDirWord)
        
    
    def load_data(self, rootSoundSamplePath, targetDirWord):

        print(get_immediate_subdirectories(rootSoundSamplePath))
        sound_sample_paths = get_immediate_subdirectories(rootSoundSamplePath)

        self.map_sound_sets = dict()
        self.selected_sound_set = None

        for sound_sample_path in sound_sample_paths:
            path_to_sound_sample = os.path.join(rootSoundSamplePath, sound_sample_path)
            print("\nReading in:", path_to_sound_sample)
            map_sounds_to_trials = parse_and_create_sound_trials(path_to_sound_sample)
            sound_set = SoundSet(sound_sample_path, map_sounds_to_trials)
            self.map_sound_sets[sound_set.get_base_path()] = sound_set
            if targetDirWord in sound_sample_path:
                    self.selected_sound_set = sound_set

        if self.selected_sound_set is not None:
            print("\nThe selected sound set:", self.selected_sound_set)
        

    def get_sound_set_with_str(self, targetStr):
        for base_path, sound_set in self.map_sound_sets.items():
            if targetStr in base_path:
                #print("set: ",sound_set)
                return sound_set
        return None


In [1]:
# Playing with FFT specgrams


In [97]:
#%%time

# This is the simplest possible SVM using only a few features but gives you a sense of the overall approach
# Some nice resources:
#  - A very simple classification example using scikit: 
#     https://dbaumgartel.wordpress.com/2014/03/10/a-scikit-learn-example-in-10-lines/
#  - A nice video overview of SVM: https://youtu.be/N1vOgolbjSc
#  - Official sci-kit learn: http://scikit-learn.org/stable/modules/svm.html

from sklearn import svm
from sklearn.preprocessing import StandardScaler
import itertools
from scipy.stats import kurtosis, skew

# Returns a feature vectof for the given trial
def extract_features_example(sample):

    # Play around with features to extract and use in your model
    # Brainstorm features, visualize ideas, try them, and iterate
    # This is likely where you will spend most of your time :)
    # This is the "feature engineering" component of working in ML
    features = []

    # Length
    #features.append(len(trial.sound))

    features.append(np.mean(sample.sound))

    return features

class TrainedModel:

    
    def __init__(self, selectedSet, modelType = "svm", targetedString="Test"):
        self.selected_sound_set = selectedSet
        self.numSounds = self.selected_sound_set.get_num_sounds()
        self.numTrialsTotal = self.selected_sound_set.get_total_num_of_trials()


        trainingData = []
        self.classLabels = np.array([])

        # build training data for this set of folds
        #for trainingSample in self.selected_sound_set.map_sounds_to_trials:
        #    for trainingSoundName, trainingTrial in trainingSample.items():
        for trainingSoundName, trainingTrials in self.selected_sound_set.map_sounds_to_trials.items():
            for trial in trainingTrials:
                features = extract_features_example(trial)
                trainingData.append(features)
                self.classLabels = np.append(self.classLabels, trainingSoundName)

        # Playing with scaling data, code taken from 5/21 lecture slides
        self.trainingData = np.array(trainingData)
        self.scaler = StandardScaler()
        self.scaler.fit(trainingData)
        self.training_data_scaled = self.scaler.transform(trainingData)

        self._trainModel(modelType)

    def _trainModel(self, modelType):
        if modelType == "svm":
            # Here, we train SVM, the 'rbf' kernal is default
            # if you use rbf, need to set gamma and C parameters
            # play around with different kernels, read about them, and try them. What happens?
            # see: 
            # - https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html#sphx-glr-auto-examples-svm-plot-rbf-parameters-py
            # - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
            self.model = svm.SVC(kernel='linear', gamma=0.01) # kernel='rbf'
            self.model.fit(np.array(self.training_data_scaled), self.classLabels) 
        
    def classifySample(self, liveSoundSample):
        features = np.array(extract_features_example(liveSoundSample))
        features_scaled = self.scaler.transform([features])
        
        prediction = self.model.predict(features_scaled)
        return prediction
     


# Main

In [98]:
# main
map_sound_sets = MapSoundSets()
trained_model = TrainedModel(map_sound_sets.selected_sound_set, targetedString="thresholdStop")

test = SoundTrial("coffee", 0, "./SoundSamples/6secSample/CoffeeGrinder_0_captured.wav")
testLive = LiveSound(test.sound, test.sample_rate)

print(trained_model.classifySample(testLive))

['thresholdStop', '6secSample']

Reading in: ./SoundSamples/thresholdStop
Found 51 wav files in ./SoundSamples/thresholdStop
Found 8 sounds
Found 6 trials for 'GarbageDisposal'
Found 7 trials for 'MicrowaveDoorClose'
Found 6 trials for 'MicrowaveDoorOpen'
Found 6 trials for 'MicrowaveEnding'
Found 8 trials for 'Toaster'
Found 6 trials for 'FridgeDoorOpen'
Found 6 trials for 'CoffeeGrinder'
Found 6 trials for 'FridgeDoorClose'

Reading in: ./SoundSamples/6secSample
Found 49 wav files in ./SoundSamples/6secSample
Found 8 sounds
Found 6 trials for 'GarbageDisposal'
Found 6 trials for 'MicrowaveDoorClose'
Found 6 trials for 'MicrowaveDoorOpen'
Found 6 trials for 'MicrowaveEnding'
Found 7 trials for 'Toaster'
Found 6 trials for 'FridgeDoorOpen'
Found 6 trials for 'CoffeeGrinder'
Found 6 trials for 'FridgeDoorClose'

The selected sound set: 'thresholdStop' : 8 sounds and 51 total trials
['Toaster']


In [60]:


# Feature extraction testing

targeted_string = "Test"
selected_sound_set = get_sound_set_with_str(targeted_string)
sound_trials = selected_sound_set.map_sounds_to_trials


soundName = sound_trials["FridgeDoor"]
features, labels = np.empty((0,193)), np.empty(0)
for trial in soundName:
    s = trial.sound_float
    r = trial.sample_rate
    
    stft = np.abs(librosa.stft(s))
    
    mfccs = np.mean(librosa.feature.mfcc(y=s, sr=r, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=r).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(s, sr=r).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=r).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(s), sr=r).T,axis=0)
    
    print(type(tonnetz))


NameError: name 'get_sound_set_with_str' is not defined